In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import pickle
import librosa

In [2]:
# Function to extract metadata from a WAV file
def getmetadata(filename):
    y, sr = librosa.load(filename)

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)

    y_harmonic, y_percussive = librosa.effects.hpss(y)

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    metadata_dict = {'tempo': tempo, 'chroma_stft': np.mean(chroma_stft), 'rmse': np.mean(rmse),
                     'spectral_centroid': np.mean(spec_centroid), 'spectral_bandwidth': np.mean(spec_bw),
                     'rolloff': np.mean(spec_rolloff), 'zero_crossing_rates': np.mean(zero_crossing)}

    for i in range(1, 21):
        metadata_dict.update({'mfcc' + str(i): np.mean(mfcc[i - 1])})

    return list(metadata_dict.values())

# 2. Loading the dataset
df = pd.read_csv("music_classification.csv")
X = df.iloc[:, 1:28]
y = df['class_name']

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# 3. Preprocessing the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, stratify=y)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Hyperparameter tuning for models
# SVM
svm = SVC(kernel='linear', probability=True)
param_svm = {'C': [0.1, 1, 10, 100]}
svm_search = RandomizedSearchCV(svm, param_svm, n_iter=10, scoring='accuracy', cv=5, random_state=3)
svm_search.fit(X_train_scaled, y_train)
svm_best = svm_search.best_estimator_

# KNN
knn = KNeighborsClassifier()
param_knn = {'n_neighbors': range(1, 31)}
knn_search = RandomizedSearchCV(knn, param_knn, n_iter=10, scoring='accuracy', cv=5, random_state=3)
knn_search.fit(X_train_scaled, y_train)
knn_best = knn_search.best_estimator_

# Logistic Regression
logreg = LogisticRegression(max_iter=10000)
param_logreg = {'C': [0.1, 1, 10, 100]}
logreg_search = RandomizedSearchCV(logreg, param_logreg, n_iter=10, scoring='accuracy', cv=5, random_state=3)
logreg_search.fit(X_train_scaled, y_train)
logreg_best = logreg_search.best_estimator_

# XGBoost
xgbc = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
param_xgbc = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}
xgbc_search = RandomizedSearchCV(xgbc, param_xgbc, n_iter=10, scoring='accuracy', cv=5, random_state=3)
xgbc_search.fit(X_train_scaled, y_train)
xgbc_best = xgbc_search.best_estimator_

# 5. Ensemble model
ensemble = VotingClassifier(estimators=[
    ('svm', svm_best), 
    ('knn', knn_best), 
    ('logreg', logreg_best), 
    ('xgbc', xgbc_best)
], voting='soft')

ensemble.fit(X_train_scaled, y_train)

# 6. Saving the trained models using pickle
with open('pickle/ensemble_model.pkl', 'wb') as file:
    pickle.dump(ensemble, file)

# Save the scaler and label encoder as well
with open('pickle/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('pickle/label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

print("Models, scaler, and label encoder have been saved successfully.")

C:\Users\kiet6\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\kiet6\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Models, scaler, and label encoder have been saved successfully.


In [7]:
# 2. Loading the dataset
df = pd.read_csv("music_classification.csv")
X = df.iloc[:, 1:28]
y = df['class_name']

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)